In [27]:
!pip install pathway bokeh pandas numpy --quiet

In [28]:
import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

output_notebook()


In [29]:
# Define the schema for Pathway's streaming table
class ParkingLotSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str


In [30]:
# Load data into a pandas DataFrame
input_df = pd.read_csv('dataset.csv')  # Replace with your actual dataset path

# (Optional) Preview the data
input_df.head()



,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [32]:
# Convert pandas DataFrame to Pathway Table
input_table = pw.io.csv.read("dataset.csv", schema=ParkingLotSchema)

#input_table = pw.io.table_from_pandas(input_df, schema=ParkingLotSchema)


In [33]:
# Map categorical features to numerical values for modeling
def traffic_to_num(traffic):
    return {'low': 0, 'average': 0.5, 'high': 1}.get(traffic, 0)

def vehicle_type_weight(vehicle):
    return {'car': 1, 'bike': 0.7, 'truck': 1.2}.get(vehicle, 1)


In [34]:
BASE_PRICE = 10
ALPHA = 5  # Tunable parameter

def baseline_linear_price(row):
    occupancy_rate = row['Occupancy'] / row['Capacity']
    return BASE_PRICE + ALPHA * occupancy_rate


In [35]:
BETA = 1    # Queue length weight
GAMMA = 2   # Traffic penalty
DELTA = 2   # Special day bonus
EPSILON = 1 # Vehicle type weight
LAMBDA = 0.8  # Demand scaling

def demand_based_price(row):
    occupancy_rate = row['Occupancy'] / row['Capacity']
    queue = row['QueueLength']
    traffic = traffic_to_num(row['TrafficConditionNearby'])
    special = row['IsSpecialDay']
    vtype = vehicle_type_weight(row['VehicleType'])
    demand = (ALPHA * occupancy_rate +
              BETA * queue -
              GAMMA * traffic +
              DELTA * special +
              EPSILON * vtype)
    # Normalize demand to [0, 1]
    max_demand = ALPHA + BETA * 10 + DELTA + EPSILON * 1.2  # Adjust as per data
    norm_demand = demand / max_demand
    price = BASE_PRICE * (1 + LAMBDA * norm_demand)
    price = max(BASE_PRICE * 0.5, min(BASE_PRICE * 2, price))
    return price


In [36]:
def compute_distance(lat1, lon1, lat2, lon2):
    # Simple Euclidean distance (use haversine for real geo)
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

def competitive_price(row, all_rows):
    # Find nearby lots (within 1km for example)
    distances = [
        compute_distance(row['Latitude'], row['Longitude'], r['Latitude'], r['Longitude'])
        for i, r in all_rows.iterrows()
        if r['SystemCodeNumber'] != row['SystemCodeNumber']
    ]
    min_dist = min(distances) if distances else 1
    competitor_prices = [
        demand_based_price(r)
        for i, r in all_rows.iterrows()
        if r['SystemCodeNumber'] != row['SystemCodeNumber'] and
           compute_distance(row['Latitude'], row['Longitude'], r['Latitude'], r['Longitude']) < 0.01
    ]
    avg_competitor_price = np.mean(competitor_prices) if competitor_prices else BASE_PRICE
    price = demand_based_price(row)
    if row['Occupancy'] >= row['Capacity']:
        if price > avg_competitor_price:
            price = avg_competitor_price * 0.95
    elif price < avg_competitor_price:
        price = min(price * 1.05, BASE_PRICE * 2)
    return price


In [41]:
# Process the data in batch (simulate real-time streaming)
prices = []
for idx, row in input_df.iterrows():
    price1 = baseline_linear_price(row)
    price2 = demand_based_price(row)
    #price3 = competitive_price(row, input_df)
    prices.append({
        'ID': row['ID'],
        'SystemCodeNumber': row['SystemCodeNumber'],
        'Timestamp': f"{row['LastUpdatedDate']} {row['LastUpdatedTime']}",
        'Price_Linear': price1,
        'Price_Demand': price2,
       # 'Price_Competitive': price3
    })

prices_df = pd.DataFrame(prices)


In [45]:
# Plot price over time for a single parking lot
lot_id = 'BHMBCCMKT01'
lot_prices = prices_df[prices_df['SystemCodeNumber'] == lot_id]
lot_prices_unique = lot_prices.drop_duplicates(subset=['Timestamp'])

source = ColumnDataSource(lot_prices_unique)
p = figure(x_range=lot_prices_unique['Timestamp'], height=350, title=f"Real-Time Pricing for Lot {lot_id}",
           x_axis_label='Time', y_axis_label='Price ($)', sizing_mode="stretch_width")
p.line(x='Timestamp', y='Price_Linear', source=source, legend_label="Linear", line_width=2, color='blue')
p.line(x='Timestamp', y='Price_Demand', source=source, legend_label="Demand-Based", line_width=2, color='green')
#p.line(x='Timestamp', y='Price_Competitive', source=source, legend_label="Competitive", line_width=2, color='red')
p.legend.location = "top_left"
p.xaxis.major_label_orientation = 1.2

show(p)


In [47]:
print(prices_df.head(100).to_markdown(index=False))


|   ID | SystemCodeNumber   | Timestamp           |   Price_Linear |   Price_Demand |
|-----:|:-------------------|:--------------------|---------------:|---------------:|
|    0 | BHMBCCMKT01        | 04-10-2016 07:59:00 |        10.5286 |        11.1115 |
|    1 | BHMBCCMKT01        | 04-10-2016 08:25:00 |        10.5546 |        11.1229 |
|    2 | BHMBCCMKT01        | 04-10-2016 08:59:00 |        10.6932 |        11.6234 |
|    3 | BHMBCCMKT01        | 04-10-2016 09:32:00 |        10.9272 |        11.7262 |
|    4 | BHMBCCMKT01        | 04-10-2016 09:59:00 |        11.2998 |        11.7582 |
|    5 | BHMBCCMKT01        | 04-10-2016 10:26:00 |        11.5338 |        12.4324 |
|    6 | BHMBCCMKT01        | 04-10-2016 10:59:00 |        11.8977 |        13.1199 |
|    7 | BHMBCCMKT01        | 04-10-2016 11:25:00 |        12.1404 |        13.1386 |
|    8 | BHMBCCMKT01        | 04-10-2016 11:59:00 |        12.2444 |        13.1843 |
|    9 | BHMBCCMKT01        | 04-10-2016 12:29:00 |   